<a href="https://colab.research.google.com/github/sahithikodali1/Summarization-of-Biomedical-evidence/blob/Data_processing/BioASQ_bl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os

print(os.listdir("/content/drive/MyDrive/Thesis_B"))

['rouge_8b_mod.csv']


In [ ]:
DATA_DIR = "/content/drive/MyDrive/Thesis_B"

file = pd.read_csv(os.path.join(DATA_DIR,'rouge_8b_mod.csv'))
f_values = file.values
# print(f_values[:2])
# print(col_names[:10])
q_id_split = np.split(f_values, np.where(np.diff(f_values[:,0]))[0]+1)
print(q_id_split[:2])

In [3]:
#Predict train and test sizes

train_data_size = round(len(q_id_split)*0.8)
test_data_size = round(len(q_id_split) - train_data_size)
print(train_data_size)
print(test_data_size)

2594
648


In [4]:
#Shuffle qid randomly

import random
random.seed(3007)
random.shuffle(q_id_split)
print(type(q_id_split[:1]))

<class 'list'>


In [5]:
#Split train data into train and validation

train_data_beforeval = q_id_split[:2594]
f_values_train_beforeval = np.concatenate(train_data_beforeval, axis=0)
# print((f_values_train_beforeval))

q_id_split_train = np.split(f_values_train_beforeval, np.where(np.diff(f_values_train_beforeval[:,0]))[0]+1)
val_data_size = round(len(q_id_split_train)*0.2)
train_data_size = round(len(q_id_split_train) - val_data_size)
print(train_data_size)
print(val_data_size)

2075
519


In [6]:
train_data = q_id_split_train[:2075]
val_data = q_id_split_train[2075:]
test_data = q_id_split[2594:]
f_values_train = np.concatenate(train_data, axis=0)
f_values_val = np.concatenate(val_data, axis=0)
f_values_test = np.concatenate(test_data, axis=0)

In [7]:
col_names = file.columns
print(col_names)

Index(['qid', 'pubmedid', 'sentid', 'N1', 'N2', 'L', 'S4', 'SU4',
       'sentence text', 'SU4_labels'],
      dtype='object')


In [8]:
#Convert data into dataframes

train_df = pd.DataFrame(f_values_train, columns = col_names)
val_df = pd.DataFrame(f_values_val, columns = col_names)
test_df = pd.DataFrame(f_values_test, columns = col_names)

In [9]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

print(len(train_df))
print(len(val_df))
print(len(test_df))
print(train_df.columns.values)

2075
519
648
33435
8148
10161
['qid' 'pubmedid' 'sentid' 'N1' 'N2' 'L' 'S4' 'SU4' 'sentence text'
 'SU4_labels']


In [ ]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification, BertForMaskedLM
from tqdm import tqdm, trange

import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 897317.36B/s]


In [12]:
def data_grouplists(data):
  data_list = data.groupby('qid', as_index=False)['SU4','SU4_labels','sentence text'].agg(list)
  return data_list

In [13]:
train_grouped = data_grouplists(train_df)
val_grouped = data_grouplists(val_df)
test_grouped = data_grouplists(test_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [14]:
print(type(train_grouped))
print(len(train_grouped))
print(train_grouped[:2])
print(train_grouped.columns.values)

<class 'pandas.core.frame.DataFrame'>
2075
   qid  ...                                      sentence text
0    2  ...  [Papilin is an extracellular matrix glycoprote...
1    4  ...  [Osteoprotegerin (OPG) and receptor activator ...

[2 rows x 4 columns]
['qid' 'SU4' 'SU4_labels' 'sentence text']


In [15]:
#Add special tokens

def add_specialtokens(data_list):
  specialtok_sentences = []

  for sentence in data_list['sentence text']:
    each_specialtoken = []
    for each in sentence:
      each_specialtoken.append(["[CLS] " + each + " [SEP]" ])
    specialtok_sentences.append(each_specialtoken)
  return specialtok_sentences

In [16]:
train_specialtok = add_specialtokens(train_grouped)
val_specialtok = add_specialtokens(val_grouped)
test_specialtok = add_specialtokens(test_grouped)

In [17]:
print(type(train_specialtok))
print(len(train_specialtok[0]))
print(train_specialtok[:2])
print(train_specialtok[0])

<class 'list'>
11
[[['[CLS] Papilin is an extracellular matrix glycoprotein [SEP]'], ['[CLS] apilins are homologous, secreted extracellular matrix proteins which share a common order of protein domains. [SEP]'], ['[CLS] Both MIG-6 isoforms have a predicted N-terminal papilin cassette [SEP]'], ['[CLS] The TSR superfamily is a diverse family of extracellular matrix and transmembrane proteins, many of which have functions related to regulating matrix organization, cell-cell interactions and cell guidance. [SEP]'], ['[CLS]  Collagen IV, laminin, glutactin, papilin, and other extracellular matrix proteins were made primarily by hemocytes and were secreted into the medium. [SEP]'], ['[CLS] A sulfated glycoprotein was isolated from the culture media of Drosophila Kc cells and named papilin. [SEP]'], ['[CLS] Using expression analysis, we identify three genes that are transcriptionally regulated by HLH-2: the protocadherin cdh-3, and two genes encoding secreted extracellular matrix proteins, mi

In [18]:
#Tokenize sentences

def tokenize_sentences(specialtok_sentences):
  tokenized_sentences = []

  for sentence in specialtok_sentences:
    eachsent_tokenized = []
    for each in sentence:
      eachsent_tokenized.append(tokenizer.tokenize(each[0]))
    tokenized_sentences.append(eachsent_tokenized)
  return tokenized_sentences

In [19]:
train_tokenized = tokenize_sentences(train_specialtok)
val_tokenized = tokenize_sentences(val_specialtok)
test_tokenized = tokenize_sentences(test_specialtok)

In [20]:
print(type(train_tokenized[0]))
print(len(train_tokenized))
print(len(train_tokenized[0]))
print(train_tokenized[:2])
print(train_tokenized[0])

<class 'list'>
2075
11
[[['[CLS]', 'pa', '##pi', '##lin', 'is', 'an', 'extra', '##cellular', 'matrix', 'g', '##ly', '##co', '##pro', '##tein', '[SEP]'], ['[CLS]', 'api', '##lins', 'are', 'homo', '##log', '##ous', ',', 'secret', '##ed', 'extra', '##cellular', 'matrix', 'proteins', 'which', 'share', 'a', 'common', 'order', 'of', 'protein', 'domains', '.', '[SEP]'], ['[CLS]', 'both', 'mig', '-', '6', 'iso', '##forms', 'have', 'a', 'predicted', 'n', '-', 'terminal', 'pa', '##pi', '##lin', 'cassette', '[SEP]'], ['[CLS]', 'the', 'ts', '##r', 'superfamily', 'is', 'a', 'diverse', 'family', 'of', 'extra', '##cellular', 'matrix', 'and', 'trans', '##me', '##mb', '##rane', 'proteins', ',', 'many', 'of', 'which', 'have', 'functions', 'related', 'to', 'regulating', 'matrix', 'organization', ',', 'cell', '-', 'cell', 'interactions', 'and', 'cell', 'guidance', '.', '[SEP]'], ['[CLS]', 'col', '##lage', '##n', 'iv', ',', 'lam', '##ini', '##n', ',', 'g', '##lu', '##ta', '##ct', '##in', ',', 'pa', '##pi',

In [21]:
#Convert tokenized sentences to respective token ids

def token2ids(tokenized_sentences):
  # Set the maximum sequence length. 
  MAX_LEN = 512
  token_ids = []

  for tokenized_sentence in tokenized_sentences:
    tokens_to_ids = [tokenizer.convert_tokens_to_ids(each) for each in tokenized_sentence]
    tokens_to_ids = pad_sequences(tokens_to_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    token_ids.append(tokens_to_ids)
  return token_ids

In [22]:
train_tokens2ids = token2ids(train_tokenized)
val_tokens2ids = token2ids(val_tokenized)
test_tokens2ids = token2ids(test_tokenized)

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (529 > 512). Running this sequence through BERT will result in indexing errors


In [23]:
print(len(train_tokens2ids))
print(len(train_tokens2ids[0]))
print(len(train_tokens2ids[0][0]))


print(type(train_tokens2ids))
print(type(train_tokens2ids[0]))
print(type(train_tokens2ids[0][0]))


print(train_tokens2ids[0].dtype)
print(train_tokens2ids[0][0].dtype)

2075
11
512
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
int64
int64


In [24]:
# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [25]:
#Create masks for the tokenids

def create_masks(token_ids):
  attention_masks = []

  for tid in token_ids:
    eachtid_mask = []
    for each in tid:
      mask = [float(i>0) for i in each] #for each token 
      eachtid_mask.append(mask)#for each sentence
    attention_masks.append(np.array(eachtid_mask))
  return attention_masks

In [26]:
train_masks = create_masks(train_tokens2ids)
val_masks = create_masks(val_tokens2ids)
test_masks = create_masks(test_tokens2ids)

In [27]:
#Create tesnors with padding

from torch.nn.utils.rnn import pad_sequence

def padding2tensor(givenlist):
 tensors = pad_sequence([torch.tensor(x) for x in givenlist], batch_first=True)
 return tensors

In [28]:
train_tensor_inputs =  padding2tensor(train_tokens2ids)
val_tensor_inputs =  padding2tensor(val_tokens2ids)
test_tensor_inputs = padding2tensor(test_tokens2ids)

In [29]:
print(len(train_tensor_inputs))
print(len(train_tensor_inputs[0]))
print(len(train_tensor_inputs[0][0]))

print(train_tensor_inputs.shape)
print(train_tensor_inputs[0].shape)
print(train_tensor_inputs[0][0].shape)

2075
146
512
torch.Size([2075, 146, 512])
torch.Size([146, 512])
torch.Size([512])


In [30]:
train_tensor_masks =  padding2tensor(train_masks)
val_tensor_masks =  padding2tensor(val_masks)
test_tensor_masks = padding2tensor(test_masks)

In [31]:
print(len(train_tensor_masks))
print(len(train_tensor_masks[0]))
print(len(train_tensor_masks[0][0]))

print(train_tensor_masks.shape)
print(train_tensor_masks[0].shape)
print(train_tensor_masks[0][0].shape)

2075
146
512
torch.Size([2075, 146, 512])
torch.Size([146, 512])
torch.Size([512])


In [32]:
#Create labels from data

train_labels = list(train_grouped['SU4'])
val_labels = list(val_grouped['SU4'])
print(len(train_labels))
print(len(val_labels))

2075
519


In [33]:
train_tensor_labels =  padding2tensor(train_labels)
val_tensor_labels =  padding2tensor(val_labels)

In [34]:
print(len(train_tensor_labels))
print(len(train_tensor_labels[0]))

print(train_tensor_labels.shape)
print(train_tensor_labels[0].shape)

2075
146
torch.Size([2075, 146])
torch.Size([146])


In [35]:
# Create an iterator of our data with torch DataLoader 

# Select a batch size for training. 
batch_size = 16

train_data = TensorDataset(train_tensor_inputs, train_tensor_masks, train_tensor_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)

validation_data = TensorDataset(val_tensor_inputs, val_tensor_masks, val_tensor_labels)
validation_sampler = RandomSampler(validation_data) #change to seq sampler next time and try
validation_dataloader = DataLoader(validation_data, sampler = validation_sampler, batch_size = batch_size)